In [1]:
#Library imports
import re
import os
import shutil
import time
import math
import fnmatch
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import matplotlib.font_manager as fm

from IPython.display import Image, display
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from PIL import Image, ImageDraw, ImageOps

from tensorflow.keras import models, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2


#User Created functions

from cleaning_functions import *
from eda_functions import *
from modeling_functions import *
from setup_functions import *

from random_lumberjacks.src.random_lumberjacks.model.model_classes import *
from random_lumberjacks.src.random_lumberjacks.visualization.visualization_functions import *

#Notebook arguments
%matplotlib inline

In [2]:
#Without this block the Japanese font's won't display properly in Matplotlib.Set to your font directory.
extend_matplotlib_fonts("/usr/share/fonts/")

/home/anacuda/work/nyc-mhtn-ds-021720/japanese_text_classifiers/setup_functions.py:23: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  font_list = fm.createFontList(font_files)


In [3]:
#Device specific gpu deterministic arguments
from tensorflow import config as tfconfig
physical_devices = tfconfig.list_physical_devices('GPU')
tfconfig.experimental.set_memory_growth(physical_devices[0], enable=True)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [4]:
#Sets random seeds to allow for reproducable results.
from tensorflow import random as tfrandom
SEED=127
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tfrandom.set_seed(SEED)

Since there are many classes, I've imported a json that contains useful information about the characters and stored it into a DataFrame.

In [5]:
hiragana_classes = pd.read_json("choubenkyo_kivy_app/app/draw_screen/characters/hiragana.json")

# By converting the recorded stroke count to a list of integers, it will be easier to iterate through the data and remove obvious mistakes from the start.
hiragana_classes["stroke_count"] = hiragana_classes["stroke_count"].map(lambda x: [int(item[0]) for item in re.finditer("\d+", x)])

# It will be useful to have seperate dataframes to account for dropped classes whether they are the obsolete kana or the compound characters.
current_hiragana_classes = hiragana_classes[(hiragana_classes["char_id"]!= "wi") & (hiragana_classes["char_id"]!= "we")].reset_index(drop=True)
reduced_hiragana_classes = current_hiragana_classes.iloc[0:46]
compound_hiragana_classes = current_hiragana_classes.iloc[46:]

## Creating Bitmaps

The first goal is to convert all the raw coordinate data collected from the phone app into 28 x 28 greyscale images to match the other datasets. The Pillow library has a function that can render these points to a bitmap image. I've created several functions that ensure that the images are rendered to be centered/filled to the image space.

In [6]:
def render_coordinates_file_to_img(coord_list, resolution=(28, 28), stroke_width=2):
    scaled = scale_points_for_pixels(coord_list, resolution, stroke_width)
    img = Image.new('L', resolution, color=0)
    draw = ImageDraw.Draw(img)
    for coords in scaled:
        line_from_array(draw, coords, width=stroke_width)
    return img

def create_classification_dirs(path_list, labels):
    for path in path_list:
        for label in labels:
            os.makedirs(os.path.join(path, label))

Then in order to loop through and QC/arrange the data in an efficient manner a few paths need to be stored as variables.

In [7]:
raw_path = "data/choubenkyo_data/raw"
rendered_path = "data/choubenkyo_data/rasterized"
rendered_split_paths = {"train": os.path.join(rendered_path, "train"),
                        "val": os.path.join(rendered_path, "val"),
                        "test": os.path.join(rendered_path, "test")}

One major change that I made to the raw coordinate data to enhance both the lstm and the bitmap files was to standardize it to a consistent number of points along a b-spline (done using the [NURBS-Python library](https://nurbs-python.readthedocs.io)). Having a consistent shape for all of the parameters is essential for the lstm and if I were to accomplish this through downsampling the points, it would outright ruin the data in many scenarios. The B-spline made the intermediate transitions natural along a curve and while typically use of a b-spline would smooth out corner points in an obtrusive way, the real time capture of the app essentially captures more points during moments where the flow of writing is slowed down (ie. a sharp direction change). With a surplus of control points towards the corners, the smoothing is minimized in places where it wouldn't be desired.

In [75]:
# Reproducible results
np.random.seed(SEED)

#Creating the inital train, val, test directories
create_classification_dirs(rendered_split_paths.values(), reduced_hiragana_classes["char_id"])

# This is the list which will become the lstm input and labels
choubenkyo_data = []
choubenkyo_labels = []

#This list will reference the directory of origin, and the files that are generated/renamed through the train, test, split
#to external image file functions.
pathmap = []

for source in os.listdir(raw_path):
    old_dir = os.path.join(raw_path, source)
    contents = os.listdir(old_dir)
    for label in reduced_hiragana_classes["char_id"]:
        matches = fnmatch.filter(contents, f"{label}*")
        np.random.shuffle(matches)
        match_splits = (image_path_list_train_test_split(matches, .7, .1))
        new_file_counters = [len(os.path.join(split_dir, label)) for split_dir in rendered_split_paths]
        for i,(key, value) in enumerate(rendered_split_paths.items()):
            new_dir = os.path.join(value, label)
            for match in match_splits[i]:
                match_raw_path = os.path.join(old_dir, match)
                raw_strokes = load_pickle(match_raw_path)
                smoothed_strokes = parse_to_points_list(raw_strokes)
                if len(raw_strokes) not in hiragana_classes[hiragana_classes["char_id"]==label]["stroke_count"].reset_index(drop=True)[0]:
                    pathmap.append([match_raw_path, np.nan])
                    continue
                new_path = os.path.join(new_dir, f"{label}{key}{new_file_counters[i]:05}.png")
                
                #Exports images for the CNN.
                img = render_coordinates_file_to_img(smoothed_strokes, stroke_width=1)
                img = ImageOps.expand(img, 2)
                img.save(new_path)
                
                #Saves observations for the lstm
                choubenkyo_data.append(strokes_to_array(smoothed_strokes))
                choubenkyo_labels.append(label)
                
                #Saves information to the document that preserves the link to the source images.
                pathmap.append([match_raw_path, new_path])
                
                #Tracks changes to image iterator so some searches for existing directories can be avoided.
                new_file_counters[i] += 1

pathmap = pd.DataFrame(pathmap, columns=["orig_file", "new_file"])
choubenkyo_data = np.vstack(choubenkyo_data)
choubenkyo_labels = pd.Series(choubenkyo_labels)

## Modeling

### The CNN

In [8]:
train_generator = ImageDataGenerator(rescale=1./255,
                                     rotation_range=4,
                                   width_shift_range=0.1, 
                                   height_shift_range=0.1, 
                                   shear_range=0.3, 
                                   zoom_range=0.1, 
                                   horizontal_flip=False)
test_generator = ImageDataGenerator(rescale=1./255)

In [9]:
X_train_generator = train_generator.flow_from_directory(rendered_split_paths["train"], target_size=(32, 32), color_mode='grayscale', seed=SEED, batch_size = 32)
X_val_generator = test_generator.flow_from_directory(rendered_split_paths["val"], target_size=(32, 32), color_mode='grayscale', seed=SEED, batch_size = 32)
X_test_generator = test_generator.flow_from_directory(rendered_split_paths["test"], target_size=(32, 32), color_mode='grayscale', seed=SEED, batch_size = 32)

Found 657 images belonging to 46 classes.
Found 190 images belonging to 46 classes.
Found 118 images belonging to 46 classes.


In [10]:
modelkuzushiji = models.load_model('model_backups/kuzushiji/model9-21')

In [11]:
layers = modelkuzushiji.layers[:-2] + [Dense(40, "relu"), Dense(46, activation='softmax')]
compile_kwargs = {"loss":"categorical_crossentropy", "optimizer":Adam(learning_rate=0.00005), "metrics":['accuracy']}
modelc1 = generate_keras_model(Sequential(), layers, compile_kwargs)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        320       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 8, 8, 128)         7

In [12]:
for layer in modelc1.layers[0:3]:
    layer.trainable = False

In [13]:
backupsc1 = ModelCheckpoint("model_backups/choubenkyo/modelc1-{epoch:02d}", monitor="val_loss")
stopping = EarlyStopping(monitor="val_loss", min_delta=0, patience=3, restore_best_weights=True)

resultsc1 = modelc1.fit(X_train_generator, epochs=200, steps_per_epoch=20, validation_data=X_val_generator, validation_steps=5, callbacks=[backupsc1, stopping])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 20 steps, validate for 5 steps
Epoch 1/200
13/20 [==================>...........] - ETA: 0s - loss: 5.1505 - accuracy: 0.0224     WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_backups/choubenkyo/modelc1-01/assets
20/20 [==============================] - 2s 104ms/step - loss: 4.8982 - accuracy: 0.0240 - val_loss: 4.1028 - val_accuracy: 0.0125
Epoch 2/200
20/20 [==============================] - 1s 38ms/step - loss: 3.8970 - accuracy: 0.0432 - val_loss: 3.4963 - val_accuracy: 0.0875
Epoch 3/200
20/20 [==============================] - 1s 38ms/step - loss: 3.4963 - accuracy: 0.06

In [14]:
test_batch = next(X_test_generator)
image_class_evaluation(modelc1, test_batch[0], test_batch[1])

Accuracy: 0.9375
Balanced Accuracy: 0.9285714285714286


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


(0.9375, 0.9285714285714286)

### The LSTM

In [95]:
# Replaces the labels with an index.
reduced_hiragana_label_dict = {key:value for key, value in reduced_hiragana_classes.reset_index()[["char_id","index"]].to_numpy()}

In [96]:
reduced_hiragana_label_dict

{'a': 0,
 'i': 1,
 'u': 2,
 'e': 3,
 'o': 4,
 'ka': 5,
 'ki': 6,
 'ku': 7,
 'ke': 8,
 'ko': 9,
 'sa': 10,
 'si': 11,
 'su': 12,
 'se': 13,
 'so': 14,
 'ta': 15,
 'ti': 16,
 'tu': 17,
 'te': 18,
 'to': 19,
 'na': 20,
 'ni': 21,
 'nu': 22,
 'ne': 23,
 'no': 24,
 'ha': 25,
 'hi': 26,
 'hu': 27,
 'he': 28,
 'ho': 29,
 'ma': 30,
 'mi': 31,
 'mu': 32,
 'me': 33,
 'mo': 34,
 'ya': 35,
 'yu': 36,
 'yo': 37,
 'ra': 38,
 'ri': 39,
 'ru': 40,
 're': 41,
 'ro': 42,
 'wa': 43,
 'wo': 44,
 '-n': 45}